In [3]:
import pandas as pd
from pandas import read_excel
import numpy as np
import re
from re import sub
import multiprocessing
from unidecode import unidecode
import os
from time import time 
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Activation,Embedding,Flatten,Bidirectional,MaxPooling2D, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import SGD,Adam
from keras import regularizers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import h5py
import csv
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk
from bltk.langtools import Tokenizer, remove_stopwords
from bangla_stemmer.stemmer import stemmer

In [4]:
raw_train_data = pd.read_excel('train.xlsx', index_col = 0) 
raw_val_data = pd.read_excel('val.xlsx', index_col = 0)
raw_test_data = pd.read_excel('test.xlsx', index_col = 0)
print(raw_train_data.shape)
print(raw_test_data.shape)
print(raw_val_data.shape)

(4994, 3)
(625, 3)
(624, 3)


In [5]:
#The Bengali Characters

from bltk.langtools.banglachars import (vowels,
                                        vowel_signs,
                                        consonants,
                                        digits,
                                        operators,
                                        punctuations,
                                        others)
print(f'Vowels: {vowels}')
print(f'Vowel signs: {vowel_signs}')
print(f'Consonants: {consonants}')
print(f'Digits: {digits}')
print(f'Operators: {operators}')
print(f'Punctuation marks: {punctuations}')
print(f'Others: {others}')

Vowels: ['অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'ঌ', 'এ', 'ঐ', 'ও', 'ঔ']
Vowel signs: ['া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ৄ', 'ে', 'ৈ', 'ো', 'ৌ']
Consonants: ['ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', 'ড়', 'ঢ়', 'য়', 'ৎ', 'ং', 'ঃ', 'ঁ']
Digits: ['০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯']
Operators: ['=', '+', '-', '*', '/', '%', '<', '>', '×', '÷']
Punctuation marks: ['।', ',', ';', ':', '?', '!', "'", '.', '"', '-', '[', ']', '{', '}', '(', ')', '–', '—', '―', '~']
Others: ['৳', '৺', '্', 'ঀ', 'ঽ', '#', '$']


In [6]:
def preprocess_bangla_text(text):
    
    #Tokenize the text
    tokenizer = Tokenizer()
    words = tokenizer.word_tokenizer(text)
    
    #Remove Stopword
    filtered_words = remove_stopwords(words)
    
    #Stem Words:
    stmer = stemmer.BanglaStemmer()
    stemmed_words = [stmer.stem(word) for word in filtered_words]
    stemmed_text = " ".join(stemmed_words)
    
    #remove HTML tag from the text
    html_tag = re.compile(r'<.*?>')
    html_clean_text = re.sub(html_tag,'',stemmed_text)
    
    #remove url
    url_clean = re.compile(r"https://\S+|www\.\S+")
    url_clean_text = re.sub(url_clean,'',html_clean_text)
    
    #remove emojies
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U0001FAE0-\U0001FAFF"  # Symbols for Legacy Computing
                           u"\U0001FAC0-\U0001FAFF"  # Additional emoticons
                           u"\U0001F004-\U0001F0CF"  # Miscellaneous Symbols and Pictographs
                           u"\U0001F170-\U0001F251"  # Enclosed Alphanumeric Supplement
                           "]+", flags=re.UNICODE)
    emoji_clean_text = re.sub(emoji_pattern,'',url_clean_text)
    
    preprocessed_text = emoji_clean_text
    return preprocessed_text
    


In [7]:
text = "তোরা সবাই বিটিভির পর্দায় ফিরে আয়🙂তোদের জন্য মুভি নারেবাম্পার ফলনই তোদের বিনোদনের মূল মাধ্যম 🙄পোলাপান কতটা সিরিয়াস মাইরিভাবা যায় সামান্য মুভির সামান্য রিভিউ বিনিময়ে কী সাংঘাতিক গালাগাল🙄ম্যাঁ গো"

In [8]:
print(preprocess_bangla_text(text))

applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
তোরা সবা বিটিভির পর্দায় ফিরে আয়তো মুভি নারেবাম্পার ফলন তোদের বিনোদন মূল মাধ্যম পোলাপান কত সিরিয়াস মাইরিভাবা সামান্য মুভির সামান্য রিভিউ বিনিময় সাংঘাতিক গালাগালম্যাঁ গো


In [9]:
raw_train_data['clean_text'] = raw_train_data['text'].apply(preprocess_bangla_text)
raw_test_data['clean_text'] = raw_test_data['text'].apply(preprocess_bangla_text)
raw_val_data['clean_text'] = raw_val_data['text'].apply(preprocess_bangla_text)

applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fo

applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth

applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied sec

applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied first rules..
applied first rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied third rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied fourth rules..
applied third rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applie

applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied first rules..
ap

...

applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
appl

applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied second rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied first rules..
applied first rules..
applied second rules..
app

applied first rules..
applied second rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied second rules..
applied second rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied second rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules.

applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied second rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied second rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied

applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
appl

In [10]:
merged_df = pd.concat([raw_train_data, raw_test_data, raw_val_data], axis=0)

In [11]:
merged_df.shape

(6243, 4)

In [12]:
merged_df.head()

,text,classes,labels,clean_text
0,মেডিকেলে এসে ঠিক ফ্যামিলির ফটো ফ্রেম এর মতো মন...,sadness,1,মেডিকেল এসে ফ্যামিলির ফটো ফ্রেম মত মনে হত নিজে...
1,অনেক দিন পর অন্য রকম অনুভূতি হচ্ছে কেননা আমার...,sadness,1,অনেক দিন পর অন্য রকম অনুভূতি কেননা সদ্য পাশ কর...
2,আগে যতোটা বেশি পছন্দ করতাম দুশ্চরিত্রা মিথিলা ...,disgust,3,যতো বেশি পছন্দ করতাম দুশ্চরিত্ মিথিলা মুখার্জি...
3,মাঝে মাঝে মনে হয় তোমাকে চাওয়াটা অনেক ভুল হয়ত ভ...,sadness,1,মনে তোমাকে চাওয়া অনেক ভুল হয়ত ভুল তবে ভুল মাশু...
4,এখানে কাওকে কবর দিয়ো না তাহলে তোমাদের পরিবারের...,fear,5,কাওকে কবর দিয়ো তোমা পরিবার আর কেউ বেঁচে থাকবে ...


In [13]:
class_counts = merged_df.classes.value_counts()

In [14]:
class_counts

disgust     1553
sadness     1190
joy         1142
fear         872
anger        759
surprise     727
Name: classes, dtype: int64

In [36]:
import keras.utils
keras.utils.to_categorical


train1, test1 = train_test_split(merged_df,random_state=69, test_size=0.2)
training_sentences = []
testing_sentences = []



train_sentences=train1['clean_text'].values
train_labels=train1['labels'].values
for i in range(train_sentences.shape[0]): 

    x=str(train_sentences[i])
    training_sentences.append(x)
    
training_sentences=np.array(training_sentences)





test_sentences=test1['clean_text'].values
test_labels=test1['labels'].values

for i in range(test_sentences.shape[0]): 
    x=str(test_sentences[i])
    testing_sentences.append(x)
    
testing_sentences=np.array(testing_sentences)



train_labels=keras.utils.to_categorical(train_labels)


test_labels=keras.utils.to_categorical(test_labels)



print("Training Set Length: "+str(len(train1)))
print("Testing Set Length: "+str(len(test1)))
print("training_sentences shape: "+str(training_sentences.shape))
print("testing_sentences shape: "+str(testing_sentences.shape))
print("train_labels shape: "+str(train_labels.shape))
print("test_labels shape: "+str(test_labels.shape))

Training Set Length: 4994
Testing Set Length: 1249
training_sentences shape: (4994,)
testing_sentences shape: (1249,)
train_labels shape: (4994, 6)
test_labels shape: (1249, 6)


In [37]:
vocab_size = 25000
embedding_dim = 300
max_length = 100
trunc_type='post'
oov_tok = "<OOV>"

In [38]:
print(training_sentences.shape)
print(train_labels.shape)

(4994,)
(4994, 6)


In [39]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
print(len(word_index))
print("Word index length:"+str(len(tokenizer.word_index)))
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)


test_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(test_sequences,maxlen=max_length)

18642
Word index length:18642


In [40]:
print("Sentence :--> \n")
print(training_sentences[3]+"\n")
print("Sentence Tokenized and Converted into Sequence :--> \n")
print(str(sequences[3])+"\n")
print("After Padding the Sequence with padding length 100 :--> \n")
print(padded[3])

Sentence :--> 

কিছুদিন অপেক্ষা করুন দেখুন ইনজামাম হয় কিনা ওর হাত প্রচুর জোর অনায়াসে ছক্কা মারে ইউটিউবে ওর ভিডি দাখ নিত

Sentence Tokenized and Converted into Sequence :--> 

[1139, 557, 316, 2594, 8362, 4, 558, 123, 44, 1589, 1337, 3894, 8363, 2242, 1763, 123, 69, 28, 277]

After Padding the Sequence with padding length 100 :--> 

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0 1139  557  316
 2594 8362    4  558  123   44 1589 1337 3894 8363 2242 1763  123   69
   28  277]


In [41]:
print("Padded shape(training): "+str(padded.shape))
print("Padded shape(testing): "+str(testing_padded.shape))

Padded shape(training): (4994, 100)
Padded shape(testing): (1249, 100)


In [45]:
with tf.device('/gpu:0'):
    model= Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
    model.add(Conv1D(256, kernel_size=3, activation = "relu"))
    model.add(Bidirectional(LSTM(64, return_sequences=True)))
    model.add(Dropout(0.25))
    model.add(Conv1D(512, kernel_size=3, activation = "relu"))
    model.add(Bidirectional(LSTM(64)))
    model.add(Dropout(0.25))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Flatten())

    model.add(Dense(256,kernel_regularizer=regularizers.l2(0.001),activation="relu"))
    model.add(Dense(6, activation='softmax'))
    
    adam=Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.999,epsilon=1e-07,amsgrad=False)
    model.summary()
    model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 300)          7500000   
                                                                 
 conv1d_4 (Conv1D)           (None, 98, 256)           230656    
                                                                 
 bidirectional_4 (Bidirecti  (None, 98, 128)           164352    
 onal)                                                           
                                                                 
 dropout_4 (Dropout)         (None, 98, 128)           0         
                                                                 
 conv1d_5 (Conv1D)           (None, 96, 512)           197120    
                                                                 
 bidirectional_5 (Bidirecti  (None, 128)               295424    
 onal)                                                

In [46]:
history=model.fit(padded,train_labels,epochs=25,batch_size=512,validation_data=(testing_padded,test_labels),use_multiprocessing=True, workers=8)

Epoch 1/25
10/10 [==============================] - 66s 6s/step - loss: 1.9177 - accuracy: 0.2459 - val_loss: 1.9025 - val_accuracy: 0.2402
Epoch 2/25
10/10 [==============================] - 60s 6s/step - loss: 1.8605 - accuracy: 0.2509 - val_loss: 1.8059 - val_accuracy: 0.2402
Epoch 3/25
10/10 [==============================] - 61s 6s/step - loss: 1.6245 - accuracy: 0.3310 - val_loss: 1.6988 - val_accuracy: 0.3018
Epoch 4/25
10/10 [==============================] - 61s 6s/step - loss: 1.3642 - accuracy: 0.4588 - val_loss: 1.8984 - val_accuracy: 0.3603
Epoch 5/25
10/10 [==============================] - 58s 6s/step - loss: 1.0882 - accuracy: 0.5433 - val_loss: 1.8829 - val_accuracy: 0.3515
Epoch 6/25
10/10 [==============================] - 61s 6s/step - loss: 0.8710 - accuracy: 0.6205 - val_loss: 2.0965 - val_accuracy: 0.3651
Epoch 7/25
10/10 [==============================] - 61s 6s/step - loss: 0.6972 - accuracy: 0.7038 - val_loss: 2.6781 - val_accuracy: 0.3635
Epoch 8/25
10/10 [==

In [47]:
#accuracy calculation
loss_and_metrics = model.evaluate(padded,train_labels,batch_size=256)
print("The train accuracy is: "+str(loss_and_metrics[1]))
loss_and_metrics = model.evaluate(testing_padded,test_labels,batch_size=256)
print("The test accuracy is: "+str(loss_and_metrics[1]))

20/20 [==============================] - 14s 684ms/step - loss: 0.0520 - accuracy: 0.9964
The train accuracy is: 0.9963956475257874
5/5 [==============================] - 4s 689ms/step - loss: 3.8173 - accuracy: 0.4748
The test accuracy is: 0.47477981448173523
